# Spotipy development notebook 

This notebook does the following: 
1. Connect and Call spotify API
    * Authentication to the Spotify API
    * Extracting the refresh TOEKN from Spotify server
    * Call spotify API to get recently played / specific playlist
2. Format to df
3. Save to local sqlite3 or parquet

In [1]:
import json
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
from datetime import datetime
import pandas as pd

from dotenv import load_dotenv

In [6]:
# Read the environment variables
load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

## Connect and Call spotify API

### a) Authentication to the Spotify API

In [3]:
# Define the desired scopes as a list
scopes = ["user-read-recently-played", "user-library-read"]

# Initialize the SpotifyOAuth object with your client ID, client secret, and desired scopes
sp_oauth = SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri='http://localhost:8888/callback',
    scope=' '.join(scopes)
)

# # # Generate the authorization URL
# auth_url = sp_oauth.get_authorize_url()

# # Redirect the user to the generated auth_url to start the authorization process
# print(f"Click the following link to authorize your application: {auth_url}")

In [ ]:
# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)
sp = spotipy.Spotify(client_credentials_manager = sp_oauth)

### b) Extracting the refresh TOEKN from Spotify server

In [5]:
# Get the access token
token_info = sp_oauth.get_access_token(as_dict=True)

# The refresh token is available in the token_info dictionary
refresh_token = token_info['refresh_token']

/var/folders/0x/3ms0bnbn6196rf0m4bwzwdrm0000gn/T/ipykernel_17497/3576782162.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(as_dict=True)


### c) Call spotify API to get recently played / specific playlist

In [10]:
# GET the recently played tracks
recently_played = sp.current_user_recently_played()

In [ ]:
#Extract playlist data
playlists = sp.user_playlists('spotify')
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXcDYGt49X0ozW"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

spotify_data = sp.playlist_tracks(playlist_URI)   

## Format to df

In [11]:
## Save the json to df

song_names = []
artist_names = []
played_at_list = []
timestamps = []

# Extracting only the relevant bits of data from the json object      
for song in recently_played["items"]:
    song_names.append(song["track"]["name"])
    artist_names.append(song["track"]["album"]["artists"][0]["name"])
    played_at_list.append(song["played_at"])
    timestamps.append(song["played_at"][0:10])
    
# Prepare a dictionary in order to turn it into a pandas dataframe below       
song_dict = {
    "song_name" : song_names,
    "artist_name": artist_names,
    "played_at" : played_at_list,
    "timestamp" : timestamps
}

song_df = pd.DataFrame(song_dict, columns = ["song_name", "artist_name", "played_at", "timestamp"])

In [12]:
song_df.head()

,song_name,artist_name,played_at,timestamp
0,Somebody Said,Buchs,2023-10-24T19:04:08.449Z,2023-10-24
1,5 feet away,rayn,2023-10-24T19:02:05.727Z,2023-10-24
2,Lonely,Paul Schulze,2023-10-24T18:58:52.523Z,2023-10-24
3,I Like Me Better,EFA,2023-10-24T18:56:38.735Z,2023-10-24
4,On Me,Van Herpen,2023-10-24T18:51:33.512Z,2023-10-24


## Save to local sqlite3 or parquet

In [16]:
import sqlite3

conn = sqlite3.connect('spotify.db')
cursor = conn.cursor()

# # Get the path to the database file
# cursor.execute("PRAGMA database_list;")
# database_path = cursor.fetchone()[2]

# print("Database path:", database_path)

# Use the to_sql method to write the DataFrame to a new table in the database
table_name = 'recently_played'  # Name for the new table in the database
song_df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the database connection
conn.close()

Database path: /Users/yusali/dev/ETL_Spotify/notebook/spotify.db


In [18]:
conn = sqlite3.connect('spotify.db')
df = pd.read_sql('select * from {}'.format(table_name), conn)
conn.close()

In [25]:
df.to_parquet('../data/recently_payed.parquet', partition_cols=['timestamp', 'artist_name'])